In [52]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings(action="ignore")

In [53]:
df=pd.read_csv(r"C:\Users\USER\Desktop\ROP_Prediction-main\ROP_Prediction-main\welldata_NonOutlier.csv")
df.head()

,Depth(m),weight on bit (kg),Rotary Speed (rpm),Pump Press (KPa),Temp In(degC),Flow In(liters/min),Flow Out %,WH Pressure (KPa),ROP(1 m)
0,25.96,0.0,151.09,3197.35,32.71,4200.22,101.61,-8485.61,17.31
1,26.27,0.0,151.09,3168.46,32.68,4195.60,101.70,-8485.61,18.63
2,26.59,0.0,61.54,3134.26,32.73,4174.82,96.81,-8485.61,5.66
3,26.90,0.0,60.23,3110.61,32.86,4167.89,96.67,-8485.61,11.67
4,27.24,0.0,61.75,3144.74,32.88,4173.26,96.63,-8485.61,120.20


In [75]:
df.max()

Depth(m)                2296.94
weight on bit (kg)     21337.87
Rotary Speed (rpm)       178.86
Pump Press (KPa)       15171.96
Temp In(degC)             63.51
Flow In(liters/min)     5864.13
Flow Out %               111.21
WH Pressure (KPa)        120.04
ROP(1 m)                 274.75
dtype: float64

In [55]:
X = df[['Depth(m)', 'weight on bit (kg)', 'Rotary Speed (rpm)', 'Pump Press (KPa)',
        'Flow In(liters/min)','WH Pressure (KPa)','Flow Out %']]
y = df['ROP(1 m)']

In [56]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=87)

In [57]:
from sklearn.metrics import mean_squared_error
from mealpy.evolutionary_based.GA import BaseGA, MultiGA, EliteSingleGA, SingleGA
from mealpy.utils.problem import Problem
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

In [58]:
import warnings
warnings.filterwarnings(action="ignore")
xgb_model = XGBRegressor(
        objective = 'reg:squarederror',
        colsample_bytree = 0.5,
        learning_rate = 0.02,
        max_depth = 10,
        min_child_weight = 1,
        n_estimators = 1000,
        subsample = 0.7)

%time xgb_model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_train, y_train)], verbose=False)

y_pred_xgb = xgb_model.predict(X_test)
pred = xgb_model.predict(X_train)
pred_df = xgb_model.predict(X)

test_score  = r2_score(y_test, y_pred_xgb)
train_score = r2_score(y_train,pred)
df_score =  r2_score(y,pred_df)

print("TEST R2 SCORE: ", test_score)
print("TRAIN R2 SCORE: ", train_score)
print("DATA R2 SCORE: ", df_score)

Wall time: 7.19 s
TEST R2 SCORE:  0.8438386326741589
TRAIN R2 SCORE:  0.9988374772777102
DATA R2 SCORE:  0.9710026056970794


In [59]:
# Create a function to be optimized by the  algorithms
def obj_fun(X):
    results = xgb_model.predict(np.array(X).reshape(1, -1)) # Reshape X to have one sample and as many features as needed
    return -results  # Negative to maximize the prediction (Yield)

In [61]:
problem_dict1 = {
    "fit_func": obj_fun,
    "lb": [df['Depth(m)'].min(), df['weight on bit (kg)'].min(),df['Rotary Speed (rpm)'].min(), 
           df['Pump Press (KPa)'].min(),df["Flow In(liters/min)"].min(),df["WH Pressure (KPa)"].min(),df["Flow Out %"].min()],
    "ub": [df['Depth(m)'].max(), df['weight on bit (kg)'].max(),df['Rotary Speed (rpm)'].max(), 
           df['Pump Press (KPa)'].max(),df["Flow In(liters/min)"].max(),df["WH Pressure (KPa)"].max(),df["Flow Out %"].max()],
    "minmax": "min",
}

In [122]:
epoch = 100
pop_size = 1000
pc = 0.95
pm = 0.05

In [123]:
model = BaseGA(epoch, pop_size, pc, pm)

In [124]:
best_position, best_fitness = model.solve(problem_dict1)

2023/08/05 06:13:35 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: Solving single objective optimization problem.
2023/08/05 06:13:39 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 1, Current best: -166.76962280273438, Global best: -166.76962280273438, Runtime: 2.24610 seconds
2023/08/05 06:13:41 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 2, Current best: -184.15711975097656, Global best: -184.15711975097656, Runtime: 2.25464 seconds
2023/08/05 06:13:44 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 3, Current best: -204.35910034179688, Global best: -204.35910034179688, Runtime: 2.28686 seconds
2023/08/05 06:13:46 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 4, Current best: -209.20994567871094, Global best: -209.20994567871094, Runtime: 2.16098 seconds
2023/08/05 06:13:48 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 5, Current best: -223.79086303710938, Global best: -223.7908630

2023/08/05 06:15:29 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 45, Current best: -270.92181396484375, Global best: -270.92181396484375, Runtime: 2.42146 seconds
2023/08/05 06:15:31 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 46, Current best: -270.92181396484375, Global best: -270.92181396484375, Runtime: 2.33631 seconds
2023/08/05 06:15:34 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 47, Current best: -270.92181396484375, Global best: -270.92181396484375, Runtime: 2.51020 seconds
2023/08/05 06:15:37 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 48, Current best: -270.92181396484375, Global best: -270.92181396484375, Runtime: 3.38342 seconds
2023/08/05 06:15:41 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 49, Current best: -270.92181396484375, Global best: -270.92181396484375, Runtime: 3.72923 seconds
2023/08/05 06:15:44 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem

2023/08/05 06:17:19 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 90, Current best: -271.1427917480469, Global best: -271.1427917480469, Runtime: 2.15958 seconds
2023/08/05 06:17:21 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 91, Current best: -271.1427917480469, Global best: -271.1427917480469, Runtime: 2.13427 seconds
2023/08/05 06:17:24 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 92, Current best: -271.1427917480469, Global best: -271.1427917480469, Runtime: 2.22704 seconds
2023/08/05 06:17:26 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 93, Current best: -271.1427917480469, Global best: -271.1427917480469, Runtime: 2.15470 seconds
2023/08/05 06:17:28 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 94, Current best: -271.1427917480469, Global best: -271.1427917480469, Runtime: 2.19060 seconds
2023/08/05 06:17:30 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch

In [125]:
print("Best Solution:")
print("Depth(m):", best_position[0])
print("weight on bit (kg):", best_position[1])
print("Rotary Speed (rpm):", best_position[2])
print("Pump Press (KPa):", best_position[3])
print("Flow In(liters/min):", best_position[4])
print("WH Pressure (KPa):", best_position[5])
print("Flow Out %:", best_position[6])
print("Best Fitness:", -best_fitness)  # Convert back to positive as it was negated for maximization

Best Solution:
Depth(m): 47.34297225851324
weight on bit (kg): 29.758632578235083
Rotary Speed (rpm): 119.14100211223189
Pump Press (KPa): 2937.165247233609
Flow In(liters/min): 4101.155682343371
WH Pressure (KPa): -8490.306494267877
Flow Out %: 91.72246945361596
Best Fitness: 280.23468017578125
